**Basic idea**  
The problem is to find the angle $\theta$
<img src="./images/EPI.jpg" alt="Drawing" style="width: 600px;"/>
We can pick a parallelogram region at the left side of the black line and a parallelogram of the same size at the right side.
<img src="./images/parallelogram1.jpg" alt="Drawing" style="width: 600px;"/>
Each parallelogram has a distribution of pixel values.
The maximum of the distance between the two distributions occur when we happen to pick the two parallelograms using the "correct" $\theta$
<img src="./images/paralellogram2.png" alt="Drawing" style="width: 600px;"/>

Deprecated
:(We define the center point of the parallelogram as the refer-ence point, and then the orientation of the center line that passesthrough the center point is the disparity to be estimated. The cen-ter lines with different orientations, divide the window into twoparts of the same size. The correct line, indicating the disparity in-formation, can be figured out by finding the maximum distance between the distributions of pixel values on either side of thelines.)

**Measure the difference between the pixels of two parallelograms (Build initial volumes)**  
$\chi^2$ is used that $$\chi^2(g_\theta,h_\theta)=\sum_i\frac{(g_\theta(i)-h_\theta(i))^2}{g_\theta(i)+h_\theta(i)}$$ 
where $g_\theta$ and $h_\theta$ are the histograms of the two parallelograms.
If the $\chi^2$ distance is large, it indicates that the two parallelograms are different.  

Suppose there are $N$ possible $\theta$, then for a target pixel, $N$ distance can be calculated from an EPI and the $N$ distance forms a volume.
The $\theta$ gives the largest distance in the volume can be estimated as the $\theta$ for the target pixel.
However, we can get better result becasue we can get two EPIs for a target pixel (for a target pixel, we can slice at horizontal or vertical direction from the sub-aperture array).
The two volume from the two EPIs can be blended into one, where the blending weight is as the following
$$c = \exp(\frac{\bar{d}/d_{max}}{2\sigma^2}),$$
where $d_{max}=max_{d}d(\theta)$ and $d=\sum_\theta d(\theta)$.
<!---
$$d_{u,v}(x,y,\theta)=c_{y,v^*}(x,u^*)d_{y,v^*}(x,u^∗,θ)+c_{x,u^∗}(y,v^∗)d_{x,v^∗}(y,v^∗,\theta)$$
-->

Before we estimate $\theta$ from the volume, we can do some optimizations on the volume "image" (where a "pixel" is a volume).

**Optimization**    
The calculated distance volumes at each possible depth labelare filtered using guided filter[29], which can perform as an edge-preserving smoothing operator. The weight in the filter is largewhen the pixel has approximate color and is small otherwise. The guided filter has been widely used in labeling problems[16]for fast filtering of cost volumes.  
[29]K. He, J. Sun, X. Tang, Guided image filtering, in: European Conference on Com-puter Vision (ECCV), Springer, 2010, pp. 1–14.  
[16]C. Rhemann, A. Hosni, M. Bleyer, C. Rother, M. Gelautz, Fast cost-volume fil-tering for visual correspondence and beyond, in: Proceedings of IEEE Con-ference on Computer Vision and Pattern Recognition (CVPR), 2011, pp. 3017–3024.  

For synthetic images, the disparities are smooth enough afterguided filtering.   
However, for Lytro images with lots of noises andaliasing, the disparity map is still too noisy. Here, we recommend further using disparity optimization and enhancement method like Jeon et al.[23]. The initial disparity after guided filter is then op-timized using weighted median filter[30],graphcuts[31], and fi-nally the iterative refinement[32].  
[23]H.-G. Jeon, J. Park, G. Choe, J. Park, Y. Bok, Y.-W. Tai, I.S. Kweon, Accurate depthmap estimation from a lenslet light field camera, in: Proceedings of IEEE Inter-national Conference on Computer Vision and Pattern Recognition (CVPR), 2015,IEEE, 2015, pp. 1547–1555.  
[30]Z. Ma, K. He, Y. Wei, J. Sun, E. Wu, Constant time weighted median filtering forstereo matching and beyond, in: Proceedings of IEEE International Conferenceon Computer Vision (ICCV), 2013, IEEE, 2013, pp. 49–56.  
[31]V. Kolmogorov, R. Zabih, Multi-camera scene reconstruction via graph cuts,in: European Conference on Computer Vision (ECCV), Springer, 2002, pp. 82–96.  
[32]Q. Yang, R. Yang, J. Davis, D. Nistér, Spatial-depth super resolution for rangeimages, in: Proceedings of IEEE Conference on Computer Vision and PatternRecognition, 2007. CVPR’07., IEEE, 2007, pp. 1–8.

**Pseudo code**  
1. Build initial volumes:  
Focusing on a single pixel of a sub-aperture image, slice two (horizontal and vertical) EPIs from the sub-aperture image array.  
For each of the two EPIs  
&emsp; For each possible $\theta$ (suppose $N$ possible $\theta$)   
&emsp;&emsp; Calculate the distance (difference) between the two parallograms  
&emsp; Stack the $N$ distances into one volume  
&emsp; EndFor  
EndFor  
Calculate the weights used for mixing by $c = \exp(\frac{\bar{d}/d_{max}}{2\sigma^2})$ as in optimization part.  
Mix the two volume into one.  
2. Optimzaiton:   
Now for each pixel, we have built a volume. We smooth the volume "image" (where a "pixel" is a volume) by the follwoing techniques      
Guided filter[29], Weighted median filter[30], Graphcuts[31], The iterative refinement[32].  
3. For each pixel, pick the $\theta$ that gives the max distance.   

**Appendix: Weights of pixels of the parallelogram**  
(I think this part is a little trivial compared to other parts, so I write it as an Appendix.)  
The pixels in the parallelogram have different weight, defined by
$$w_{\theta}(i, j)=c\cdot d_{\theta}(i, j)\cdot \exp({\frac{-d^2_{\theta}(i, j)}{2\alpha}})$$
$$d_{\theta}(i, j)=i-(x_r+(j-u_r)\cdot \tan\theta)$$  
where $d_{\theta}(i, j)$ is the distance betweenthe pixel and the matching point P in the same row,  
$(x_r+(j-u_r)\cdot \tan\theta)$ is the hypothetical matching line’s position in the rowj,  
$\alpha$ is a scale parameter,  
$c$ is a normalizing constant.   
<img src="./images/x_distance.png" alt="Drawing" style="width: 600px;"/>
    From the $\exp({\frac{-d^2_{\theta}(i, j)}{2\alpha}})$ term, we can say that a parallelogram is about width $3\alpha$ because: the Empirical Rule states that almost all (99.70%) data lies within 3 standard deviations of the mean for a normal distribution.